<a href="https://colab.research.google.com/github/utkarshkant/Bank-Complaints-Text-Classification/blob/main/Text_Classification_Bank_Complaints_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install models pre-trained on the banking complaints dataset
- NLP model building: 

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1m1bVxlUjrJ_tmWApYJHlk2q5bikGyIxr' -O complaints.booster

--2020-10-07 17:53:55--  https://docs.google.com/uc?export=download&id=1m1bVxlUjrJ_tmWApYJHlk2q5bikGyIxr
Resolving docs.google.com (docs.google.com)... 108.177.97.139, 108.177.97.113, 108.177.97.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q54ebnpd93j3v2tp4vbvjig8ofdud6bb/1602093225000/00448151577512439380/*/1m1bVxlUjrJ_tmWApYJHlk2q5bikGyIxr?e=download [following]
--2020-10-07 17:53:56--  https://doc-04-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/q54ebnpd93j3v2tp4vbvjig8ofdud6bb/1602093225000/00448151577512439380/*/1m1bVxlUjrJ_tmWApYJHlk2q5bikGyIxr?e=download
Resolving doc-04-8s-docs.googleusercontent.com (doc-04-8s-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting to doc-04-8s-docs.googleusercontent.com (doc-04-8s-d

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1WURdboJjF27g9bZG_CGCCGSYWi0EvxJR' -O tfvectroizer.pkl

--2020-10-07 17:53:58--  https://docs.google.com/uc?export=download&id=1WURdboJjF27g9bZG_CGCCGSYWi0EvxJR
Resolving docs.google.com (docs.google.com)... 108.177.97.138, 108.177.97.101, 108.177.97.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.97.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mko83lnsaamt9bdbdviq26dejf5rq75d/1602093225000/00448151577512439380/*/1WURdboJjF27g9bZG_CGCCGSYWi0EvxJR?e=download [following]
--2020-10-07 17:53:59--  https://doc-0s-8s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mko83lnsaamt9bdbdviq26dejf5rq75d/1602093225000/00448151577512439380/*/1WURdboJjF27g9bZG_CGCCGSYWi0EvxJR?e=download
Resolving doc-0s-8s-docs.googleusercontent.com (doc-0s-8s-docs.googleusercontent.com)... 74.125.204.132, 2404:6800:4008:c04::84
Connecting to doc-0s-8s-docs.googleusercontent.com (doc-0s-8s-d

In [3]:
# directory contents
!ls

complaints.booster  drive  sample_data	tfvectroizer.pkl


In [27]:
# standard imports
import pandas as pd
import numpy as np
import re
import nltk
import joblib
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

# configurations
pd.set_option('display.max_colwidth', -1)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if sys.path[0] == '':


True

In [5]:
# load dataset
df= pd.read_csv("https://github.com/utkarshkant/Bank-Complaints-Text-Classification/blob/main/consumer_compliants.zip?raw=true",
                compression='zip', 
                sep=",", 
                quotechar='"')

In [6]:
# preview dataset
df.head(3)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Company has responded to the consumer and the CFPB and chooses not to provide a public response,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949


In [7]:
# size of dataset
print(f"# rows: {df.shape[0]}")
print(f"# cols: {df.shape[1]}")

# rows: 57453
# cols: 18


In [8]:
# take a small data sample
df = df.sample(10000)

In [9]:
# create a copy
complaints_df = df[['Consumer complaint narrative', 'Product']].rename(columns={'Consumer complaint narrative':'complaints'})

In [10]:
# multi class classification >> target classes
df['Product'].value_counts()

Debt collection                3782
Credit card or prepaid card    2326
Mortgage                       1698
Checking or savings account    1198
Student loan                   520 
Vehicle loan or lease          476 
Name: Product, dtype: int64

In [11]:
# map target variables
target = {0:"Debt collection",
          1:"Credit card or prepaid card",
          2:"Mortgage",
          3:"Checking or savings account",
          4:"Student loan",
          5:"Vehicle loan or lease"}

In [12]:
# preview data
complaints_df.head(3)

,complaints,Product
50173,"I'm writing on behalf of my father who does not owe anything to anyone. The company named Penn Credit Corp has been harrassing him for months. These are the dates and times this phone number XXXX has called him at his home : XX/XX/XXXX - XXXX, XXXX, XXXX ; XX/XX/XXXX - XXXX, XXXX ; XX/XX/XXXX - XXXX, XXXX, XXXX ; XX/XX/XXXX - XXXX ; XX/XX/XXXX - XXXX, XXXX, XXXX ; XX/XX/XXXX - XXXX, XXXX ; XX/XX/XXXX - XXXX, XXXX, XXXX If you would please them stop we would appreciate it!",Debt collection
56108,"I started using Bank Of America Travel card because their travel rewards program on XX/XX/2019. According to their rules and stated on advertisements, you earn double points when you book travel at Bank of America Travel center. I purchased an XXXX XXXX ticket on XX/XX/2019 on the Bank of America Travel center website and they never credited the rewards. ( see attached receipt ) .They credited the normal charge rewards as if i had purchased anything and outside the Travel Center as if I had purchased food or anything not at Travel center. I called in XXXX, then twice in XXXX, every time I was on the phone for an hour or more ( see attached ). I provided copies of THEIR website confirming the purchase, and emailed documents as requested by them. Every time they say they will credit the rewards in 7 to 10 days. This I believe is a fraudulent practice basically getting customers to get tired of calling. I wonder how many victims of this Bank of America fraud are there, but certainly something very wrong is going with this rewards program. They asked for documents of their own website and their credit card and I have provided three times. It is not a lot of money in cash value of XXXX points, about {$58.00} but certainly is theft by Bank of America. Probably they have been stealing millions from other customers not checking the full credit of their rewards.",Credit card or prepaid card
46623,"This is follow up for complain XXXX Bank of America has NOT address my concern and give me any update before close the complaint. \n\nYour feedback THE COMPANYS RESPONSE ADDRESSED ALL OF MY ISSUES No ADDITIONAL COMMENTS - it charge my interest about {$54.00} again on XX/XX/XXXX ; - it mentioned that I use `minimum payment` for the automatic payment. But I clearly remember I used `statement balance`, with which I should not be charged with any interest. \nI UNDERSTAND THE COMPANYS RESPONSE TO MY COMPLAINT No ADDITIONAL COMMENTS it mentioned that I used `minimum payment` for the automatic payment. But I clearly remember I used `statement balance`, with which I should not be charged with any interest. Please provide evidence that I used `minimum payment` for the automatic payment THE COMPANY DID WHAT THEY SAID THEY WOULD DO WITH MY COMPLAINT No ADDITIONAL COMMENTS The company promised me in the phone call, that they will : - refund all my interest charge ; - fix the auto pay settings. Obviously it is not true, as I was charged interest of {$54.00} dollar again on XX/XX/XXXX.",Credit card or prepaid card


In [13]:
# stemming
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = set(nltk.corpus.stopwords.words('english'))

In [14]:
# UDF for tokenization
def tokenize(text):
  # tokenization
  tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+', '', word.strip('Xx/'))))]
  tokens = map(str.lower, tokens)
  # stemming
  stems = [stemmer.stem(item) for item in tokens if (item not in stop_words)]
  return stems

In [15]:
# load vectorizer
tfvectorizer = joblib.load("/content/tfvectroizer.pkl")

# Predicting in batch

In [18]:
# get feature names
test_transformed = tfvectorizer.transform(complaints_df['complaints'])
tfvectorizer.get_feature_names()

['abil',
 'abl',
 'absolut',
 'abus',
 'accept',
 'access',
 'access account',
 'accord',
 'account',
 'account account',
 'account balanc',
 'account bank',
 'account call',
 'account close',
 'account credit',
 'account inform',
 'account number',
 'account open',
 'account report',
 'account would',
 'accru',
 'acct',
 'accur',
 'acknowledg',
 'act',
 'action',
 'activ',
 'actual',
 'ad',
 'addit',
 'address',
 'adjust',
 'admit',
 'advanc',
 'advantag',
 'advertis',
 'advis',
 'affect',
 'affidavit',
 'afford',
 'agenc',
 'agent',
 'agre',
 'agreement',
 'alert',
 'alleg',
 'alleg debt',
 'alli',
 'allow',
 'almost',
 'along',
 'alreadi',
 'also',
 'although',
 'alway',
 'america',
 'american',
 'american express',
 'amount',
 'amount owe',
 'and/or',
 'annual',
 'anoth',
 'answer',
 'anyon',
 'anyth',
 'apart',
 'appar',
 'appeal',
 'appear',
 'appli',
 'applic',
 'apprais',
 'appreci',
 'approv',
 'approxim',
 'around',
 'arrang',
 'arriv',
 'ask',
 'assess',
 'asset',
 'assign',

In [17]:
test_transformed.A

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0511934 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.12694687, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.13925514, ..., 0.        , 0.        ,
        0.        ]])

In [19]:
# load classifier model
xgb_clf = xgb.Booster({"nthread":4})    # create a booster object with 4 parallel threads
xgb_clf.load_model("/content/complaints.booster")

In [22]:
# make predictions
predictions = xgb_clf.predict(xgb.DMatrix(test_transformed))
predictions   # we can take argmax to make final prediction

array([[0.25693992, 0.14904048, 0.27273032, 0.07632955, 0.10630947,
        0.13865027],
       [0.0117942 , 0.8821151 , 0.00792279, 0.07397688, 0.00978895,
        0.01440206],
       [0.06325704, 0.28220764, 0.04745027, 0.06740045, 0.02794869,
        0.511736  ],
       ...,
       [0.0063159 , 0.6489839 , 0.00426962, 0.32907858, 0.00414515,
        0.00720684],
       [0.94813305, 0.01740721, 0.0075771 , 0.00734926, 0.00770181,
        0.01183155],
       [0.94863945, 0.01286931, 0.0099252 , 0.00843967, 0.00758523,
        0.01254113]], dtype=float32)

# Single Instance Prediction

In [26]:
input_text = ["I have a federal student loan and there is supposed to be a zero percent interest rate"]
test_transformed = tfvectorizer.transform(input_text)
final_class = xgb_clf.predict(xgb.DMatrix(test_transformed))
print(final_class)
print(target.get(final_class.argmax(axis=1)[0]))

[[0.01438424 0.00482566 0.01117877 0.00548103 0.9457072  0.01842318]]
Student loan


# Model Evaluation
Here we will compare the predicted vs the true values in matrices

In [29]:
reverse_target = {v:k for k,v in target.items()}
cm = confusion_matrix(complaints_df['Product'].map(reverse_target), predictions.argmax(axis=1))
print(f"Confusion Matrix:\n{cm}\n")
print("Classification Report:\n", classification_report(complaints_df['Product'].map(reverse_target), predictions.argmax(axis=1)))

Confusion Matrix:
[[3257  148   63  101   71  142]
 [ 128 1959    8  134    2   95]
 [  36   10 1533   16   33   70]
 [  27   53    7 1098    5    8]
 [  21    0    3    1  456   39]
 [  38   11    6    7    9  405]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.86      0.89      3782
           1       0.90      0.84      0.87      2326
           2       0.95      0.90      0.92      1698
           3       0.81      0.92      0.86      1198
           4       0.79      0.88      0.83       520
           5       0.53      0.85      0.66       476

    accuracy                           0.87     10000
   macro avg       0.82      0.88      0.84     10000
weighted avg       0.88      0.87      0.87     10000

